<a href="https://colab.research.google.com/github/donghui-0126/mini-project/blob/main/shoes-project/resell_regressor/img_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install Pillow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
pip install opencv-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [29]:
from PIL import Image
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.linear_model import LinearRegression
import warnings
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold


warnings.filterwarnings('ignore')     

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
df = pd.read_csv('/content/drive/MyDrive/kream_data/product_data_dpp.csv')
df.head(3)

,Unnamed: 0,product_id,img_path,brand,name,color1,color2,price_og,price_resell,n_scrap,+price,tokenized_color,tokenized_bn,drive_img_path,pixel
0,0,28029,crawling\product_crawling\image\28029.jpg,Nike,Nike Dunk Low Retro Black,WHITE,BLACK,129000,152800,123000.0,23800,[0 0 0 0 0 0 0 0 0 0 2 4 3 5],[ 0 0 0 0 0 0 0 0 0 0 0 0 0 2 4 ...,/content/drive/MyDrive/kream_data/image/28029.jpg,[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 ...
1,1,12831,crawling\product_crawling\image\12831.jpg,Nike,Nike Air Force 1 '07 Low White,WHITE,WHITE,139000,136200,151000.0,-2800,[0 0 0 0 0 0 0 0 0 0 2 4 3 4],[ 0 0 0 0 0 0 0 0 0 0 0 2 4 3 4 ...,/content/drive/MyDrive/kream_data/image/12831.jpg,[[255 255 255 ... 255 255 255]\n [255 255 255 ...
2,2,21935,crawling\product_crawling\image\21935.jpg,Nike,Nike Air Force 1 '07 WB Flax,FLAX,GUM,169000,173200,69000.0,4200,[ 0 0 0 0 0 0 0 0 0 0 2 82 3 51],[ 0 0 0 0 0 0 0 0 0 ...,/content/drive/MyDrive/kream_data/image/21935.jpg,[[255 255 255 ... 255 255 255]\n [255 255 255 ...


In [6]:
drive_img_path_list = []

for index,data in enumerate(df['img_path']):
    drive_img_path = "/content/drive/MyDrive/kream_data/image/" + (data.split("\\")[-1])
    drive_img_path_list.append(drive_img_path)

In [7]:
df["drive_img_path"] = drive_img_path_list

In [8]:
df.head(3)

,Unnamed: 0,product_id,img_path,brand,name,color1,color2,price_og,price_resell,n_scrap,+price,tokenized_color,tokenized_bn,drive_img_path,pixel
0,0,28029,crawling\product_crawling\image\28029.jpg,Nike,Nike Dunk Low Retro Black,WHITE,BLACK,129000,152800,123000.0,23800,[0 0 0 0 0 0 0 0 0 0 2 4 3 5],[ 0 0 0 0 0 0 0 0 0 0 0 0 0 2 4 ...,/content/drive/MyDrive/kream_data/image/28029.jpg,[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 ...
1,1,12831,crawling\product_crawling\image\12831.jpg,Nike,Nike Air Force 1 '07 Low White,WHITE,WHITE,139000,136200,151000.0,-2800,[0 0 0 0 0 0 0 0 0 0 2 4 3 4],[ 0 0 0 0 0 0 0 0 0 0 0 2 4 3 4 ...,/content/drive/MyDrive/kream_data/image/12831.jpg,[[255 255 255 ... 255 255 255]\n [255 255 255 ...
2,2,21935,crawling\product_crawling\image\21935.jpg,Nike,Nike Air Force 1 '07 WB Flax,FLAX,GUM,169000,173200,69000.0,4200,[ 0 0 0 0 0 0 0 0 0 0 2 82 3 51],[ 0 0 0 0 0 0 0 0 0 ...,/content/drive/MyDrive/kream_data/image/21935.jpg,[[255 255 255 ... 255 255 255]\n [255 255 255 ...


In [9]:
# pixel_list = []
# for n,url in enumerate(df['drive_img_path']):
#     img_name = (url)
#     im = Image.open(img_name)
#     pix = np.array(im)

#     print("\r{}".format(n), end="")

#     pixel_list.append(pix)
# df['pixel'] = pixel_list

In [10]:
df["pixel"].head()

0    [[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 ...
1    [[255 255 255 ... 255 255 255]\n [255 255 255 ...
2    [[255 255 255 ... 255 255 255]\n [255 255 255 ...
3    [[255 255 255 ... 255 255 255]\n [255 255 255 ...
4    [[255 255 255 ... 255 255 255]\n [255 255 255 ...
Name: pixel, dtype: object

In [11]:
df.head(3)

,Unnamed: 0,product_id,img_path,brand,name,color1,color2,price_og,price_resell,n_scrap,+price,tokenized_color,tokenized_bn,drive_img_path,pixel
0,0,28029,crawling\product_crawling\image\28029.jpg,Nike,Nike Dunk Low Retro Black,WHITE,BLACK,129000,152800,123000.0,23800,[0 0 0 0 0 0 0 0 0 0 2 4 3 5],[ 0 0 0 0 0 0 0 0 0 0 0 0 0 2 4 ...,/content/drive/MyDrive/kream_data/image/28029.jpg,[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 ...
1,1,12831,crawling\product_crawling\image\12831.jpg,Nike,Nike Air Force 1 '07 Low White,WHITE,WHITE,139000,136200,151000.0,-2800,[0 0 0 0 0 0 0 0 0 0 2 4 3 4],[ 0 0 0 0 0 0 0 0 0 0 0 2 4 3 4 ...,/content/drive/MyDrive/kream_data/image/12831.jpg,[[255 255 255 ... 255 255 255]\n [255 255 255 ...
2,2,21935,crawling\product_crawling\image\21935.jpg,Nike,Nike Air Force 1 '07 WB Flax,FLAX,GUM,169000,173200,69000.0,4200,[ 0 0 0 0 0 0 0 0 0 0 2 82 3 51],[ 0 0 0 0 0 0 0 0 0 ...,/content/drive/MyDrive/kream_data/image/21935.jpg,[[255 255 255 ... 255 255 255]\n [255 255 255 ...


In [12]:
# df.to_csv("/content/drive/MyDrive/kream_data/product_data_dpp.csv")

In [13]:
# df = pd.read_csv("/content/drive/MyDrive/kream_data/product_data_dpp.csv")

In [14]:
# pixel_data= np.array(df["pixel"])

In [15]:
# pixel_data.shape

(6103,)

In [17]:
# pixel = np.concatenate(pixel_data, axis=0).reshape((6103,256,256,1))

In [20]:
# pixel.shape

In [22]:
# np.save('/content/drive/MyDrive/kream_data/pixel_data', pixel)

In [23]:
pixel = np.load('/content/drive/MyDrive/kream_data/pixel_data.npy')
pixel.shape

(6103, 256, 256, 1)

In [42]:
from keras.models import Sequential
from keras.layers import Conv2D, Activation, MaxPooling2D, Flatten, Dense, Dropout

kfold = KFold(n_splits=5, shuffle=True)

input_shape=(256,256, 1)

model1_error = []
model2_error = []
model3_error = []

for train_index, test_index in kfold.split(pixel):
    train_X, train_y, test_X, test_y = pixel[train_index],df['price_resell'].iloc[train_index],\
                                            pixel[test_index], df['price_resell'].iloc[test_index]
    
    
    model1 = Sequential()
    model1.add(Conv2D(32, (3, 3), input_shape=input_shape))
    model1.add(Activation('relu'))
    model1.add(MaxPooling2D(pool_size=(2, 2)))

    model1.add(Conv2D(32, (3, 3)))
    model1.add(Activation('relu'))
    model1.add(MaxPooling2D(pool_size=(2, 2)))

    model1.add(Conv2D(64, (3, 3)))
    model1.add(Activation('relu'))
    model1.add(MaxPooling2D(pool_size=(2, 2)))

    model1.add(Flatten())
    model1.add(Dense(64))
    model1.add(Activation('relu'))
    model1.add(Dropout(0.5))
    model1.add(Dense(1))

    sgd = keras.optimizers.SGD(lr=0.0001)    
    model1.compile(optimizer= sgd, loss=tf.keras.losses.Huber(delta=50000))
    model1.fit(train_X, train_y, epochs=20, validation_data=(test_X,test_y))

    pred1 = model1.predict(test_X)
    mse1 = mean_squared_error(pred1, test_y, squared=False) 

print(f"MSE:{np.mean(model1_error)}")

Epoch 1/20
153/153 [==============================] - 8s 50ms/step - loss: nan - val_loss: nan
Epoch 2/20
153/153 [==============================] - 7s 47ms/step - loss: nan - val_loss: nan
Epoch 3/20
 50/153 [========>.....................] - ETA: 4s - loss: nan

KeyboardInterrupt: ignored